In [3]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-23 07:57:29--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv.4’

customer_churn.csv. 100%[===================>] 668.81K  --.-KB/s    in 0.06s   

2019-04-23 07:57:29 (10.5 MB/s) - ‘customer_churn.csv.4’ saved [684858/684858]



In [0]:
import pandas as pd
from fastai.tabular import *

In [0]:
# Preprocessing tabular data in fast.ai

df = pd.read_csv("customer_churn.csv")

In [6]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
# !pip install -q tensorflow==2.0.0-alpha0

In [0]:
# import tensorflow as tf
# from tensorflow import keras
# from sklearn.model_selection import train_test_split

In [0]:
#Here we transform all categorical variables into categories. We also replace missing values for continuous variables by the median column value and normalize those.
procs = [FillMissing, Categorify, Normalize]

In [0]:
# split our data into training and validation sets, we use valid indexes
valid_idx = range(len(df)-2000, len(df))

In [0]:
# manually split our variables into categorical and continuous variables (we can ignore the dependent variable at this stage)
dep_var = 'Exited'
cat_names = ['Geography','Gender','HasCrCard','IsActiveMember']
cont_names = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

In [12]:
# create the DataBunch that we'll use for training
data = TabularDataBunch.from_df('',df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names,cont_names=cont_names)
print(data.train_ds.cont_names)

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']


In [13]:
#We can grab a mini-batch of data and take a look (note that to_np here converts from pytorch tensor to numpy):
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[1 2 2 1]
 [3 2 1 2]
 [1 2 2 1]
 [1 2 1 2]
 [3 2 2 2]]
[[-0.115439 -0.660044  1.382361 -1.230944  0.809463  0.193689]
 [ 0.122055 -0.564908 -0.697811 -1.230944  0.809463 -0.558615]
 [ 0.586719 -0.564908  1.035666  0.959727 -0.905501 -0.798066]
 [ 0.23564   1.908638 -1.737897  1.622954  0.809463 -1.375684]
 [-1.271935  1.33782   1.035666 -1.230944 -0.905501 -0.842901]]
[0 0 0 1 0]


In [14]:
# Defining the model
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.377839,0.352965,0.853500,00:01


In [15]:
#pass the row of a dataframe as training or validation dataframe
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.6125, 0.3875]))

In [16]:
#Run 5 times we fit 5 times
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.433980,0.370726,0.850500,00:01
1,0.375503,0.363983,0.849500,00:01
2,0.359998,0.348803,0.856000,00:01
3,0.348291,0.343449,0.854500,00:01
4,0.346210,0.345075,0.862000,00:01


In [17]:
#pass the row of a dataframe as training or validation dataframe
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.6354, 0.3646]))